In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.utils import image_dataset_from_directory
import numpy as np

## Data Processing

In [ ]:
training_set=image_dataset_from_directory(
    "Dataset/train",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
validation_set=image_dataset_from_directory(
    "Dataset/val",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
training_set.class_names

In [ ]:
from collections import Counter
class_names = validation_set.class_names

In [ ]:

# Initialize a Counter for class occurrences
label_counter = Counter()

# Loop through the dataset to count labels
for images, labels in training_set:
    # labels is one-hot encoded (because you used label_mode='categorical')
    # Convert one-hot labels to class indices
    label_indices = np.argmax(labels.numpy(), axis=1)
    label_counter.update(label_indices)

# Display the count with class names
for class_index, count in label_counter.items():
    print(f"{class_names[class_index]}: {count} images")

In [ ]:
inputShape=(224, 224, 3)

In [ ]:
model=tf.keras.applications.MobileNetV3Large(
    input_shape=inputShape,
    alpha=1.0,
    minimalistic=False,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    classes=1000,
    pooling=None,
    dropout_rate=0.2,
    classifier_activation="softmax",
    include_preprocessing=True,
)

In [ ]:
# model.trainable= False ## freeze the weights of base model

In [ ]:
model2=tf.keras.models.Sequential()

In [ ]:
model2.add(tf.keras.Input(shape=inputShape))

In [ ]:
model2.add(model)

In [ ]:
model2.add(tf.keras.layers.Dense(units=4,activation='softmax'))

In [ ]:
import tensorflow_addons as tfa

metricsList = ['accuracy', tfa.metrics.F1Score(num_classes=4, average='macro')]

In [ ]:
model2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=metricsList
)

In [ ]:
'''
model2 = tf.keras.Sequential([
    tf.keras.Input(shape=inputShape),
    model,
    tf.keras.layers.Dense(units=4, activation='softmax')
])

metricsList = ['accuracy', tfa.metrics.F1Score(num_classes=4, average='macro')]

model2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=metricsList
)
'''

In [ ]:
model2.summary()

In [ ]:
history = model2.fit(x=training_set,validation_data=validation_set, epochs=15)

In [ ]:
model2.evaluate(validation_set)

In [ ]:
model2.save('Eye_disease_prediction_model.h5')

In [ ]:
import pickle
with open('training_history.pkl', 'wb') as f:
        pickle.dump(history.history, f)

In [ ]:
test_set=image_dataset_from_directory(
    "Dataset/test",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
model2.evaluate(test_set)